![Continuum Logo](./images/Continuum_Logo_0702.png)\ 

# Sympy - Symbolic algebra in Python

This curriculum builds on material by J. Robert Johansson from his "Introduction to scientific computing with Python," generously made available under a [Creative Commons Attribution 3.0 Unported License](http://creativecommons.org/licenses/by/3.0/) at https://github.com/jrjohansson/scientific-python-lectures. The Continuum Analytics enhancements use the [Creative Commons Attribution-NonCommercial 4.0 International License](https://creativecommons.org/licenses/by-nc/4.0/).

****


In [ ]:
%pylab inline

## Introduction

There are two notable Computer Algebra Systems (CAS) for Python:

* [SymPy](http://sympy.org/en/index.html) - A python module that can be used in any Python program, or in an IPython session, that provides powerful CAS features. 
* [Sage](http://www.sagemath.org/) - Sage is a full-featured and very powerful CAS enviroment that aims to provide an open source system that competes with Mathematica and Maple. Sage is not a regular Python module, but rather a CAS environment that uses Python as its programming language.

Sage is in some aspects more powerful than SymPy, but both offer very comprehensive CAS functionality. The advantage of SymPy is that it is a regular Python module and integrates well with the IPython notebook. 

In this lecture we will therefore look at how to use SymPy with IPython notebooks. If you are interested in an open source CAS environment I also recommend to read more about Sage.

To get started using SymPy in a Python program or notebook, import the module `sympy`:

In [ ]:
from sympy import *

To get nice-looking $\LaTeX$ formatted output run:

In [ ]:
init_printing()

# or with older versions of sympy/ipython, load the IPython extension
#%load_ext sympy.interactive.ipythonprinting
# or
#%load_ext sympyprinting

## Symbolic variables

In SymPy we need to create symbols for the variables we want to work with. We can create a new symbol using the `Symbol` class:

In [ ]:
x = Symbol('x')

In [ ]:
(pi + x)**2

In [ ]:
# alternative way of defining symbols
a, b, c = symbols("a, b, c")

In [ ]:
type(a)

We can add assumptions to symbols when we create them:

In [ ]:
x = Symbol('x', real=True)

In [ ]:
x.is_imaginary

In [ ]:
x = Symbol('x', positive=True)

In [ ]:
x > 0

### Complex numbers

The imaginary unit is denoted `I` in Sympy. 

In [ ]:
1+1*I

In [ ]:
I**2

In [ ]:
(x * I + 1)**2

### Rational numbers

There are three different numerical types in SymPy: `Real`, `Rational`, `Integer`: 

In [ ]:
r1 = Rational(4,5)
r2 = Rational(5,4)

In [ ]:
r1

In [ ]:
r1+r2

In [ ]:
r1/r2

## Numerical evaluation

SymPy uses a library for artitrary precision as numerical backend, and has predefined SymPy expressions for a number of mathematical constants, such as: `pi`, `e`, `oo` for infinity.

To evaluate an expression numerically we can use the `evalf` function (or `N`). It takes an argument `n` which specifies the number of significant digits.

In [ ]:
pi.evalf(n=50)

In [ ]:
y = (x + pi)**2

In [ ]:
N(y, 5) # same as evalf

When we numerically evaluate algebraic expressions we often want to substitute a symbol with a numerical value. In SymPy we do that using the `subs` function:

In [ ]:
y.subs(x, 1.5)

In [ ]:
N(y.subs(x, 1.5))

The `subs` function can of course also be used to substitute Symbols and expressions:

In [ ]:
y.subs(x, a+pi)

We can also combine numerical evolution of expressions with NumPy arrays:

In [ ]:
import numpy

In [ ]:
x_vec = numpy.arange(0, 10, 0.1)

In [ ]:
y_vec = numpy.array([N(((x + pi)**2).subs(x, xx)) for xx in x_vec])

In [ ]:
fig, ax = subplots()
ax.plot(x_vec, y_vec);

However, this kind of numerical evolution can be very slow, and there is a much more efficient way to do it: Use the function `lambdify` to "compile" a Sympy expression into a function that is much more efficient to evaluate numerically:

In [ ]:
f = lambdify([x], (x + pi)**2, 'numpy')  # the first argument is a list of variables that
                                         # f will be a function of: in this case only x -> f(x)

In [ ]:
y_vec = f(x_vec)  # now we can directly pass a numpy array and f(x) is efficiently evaluated

The speedup when using "lambdified" functions instead of direct numerical evaluation can be significant, often several orders of magnitude. Even in this simple example we get a significant speed up:

In [ ]:
%%timeit

y_vec = numpy.array([N(((x + pi)**2).subs(x, xx)) for xx in x_vec])

In [ ]:
%%timeit

y_vec = f(x_vec)

## Algebraic manipulations

One of the main uses of an CAS is to perform algebraic manipulations of expressions. For example, we might want to expand a product, factor an expression, or simply an expression. The functions for doing these basic operations in SymPy are demonstrated in this section.

### Expand and factor

The first steps in an algebraic manipulation 

In [ ]:
(x+1)*(x+2)*(x+3)

In [ ]:
expand((x+1)*(x+2)*(x+3))

The `expand` function takes a number of keywords arguments which we can tell the functions what kind of expansions we want to have performed. For example, to expand trigonometric expressions, use the `trig=True` keyword argument:

In [ ]:
sin(a+b)

In [ ]:
expand(sin(a+b), trig=True)

See `help(expand)` for a detailed explanation of the various types of expansions the `expand` functions can perform.

The opposite a product expansion is of course factoring. The factor an expression in SymPy use the `factor` function: 

In [ ]:
factor(x**3 + 6 * x**2 + 11*x + 6)

### Simplify

The `simplify` tries to simplify an expression into a nice looking expression, using various techniques. More specific alternatives to the `simplify` functions also exists: `trigsimp`, `powsimp`, `logcombine`, etc. 

The basic usages of these functions are as follows:

In [ ]:
# simplify expands a product
simplify((x+1)*(x+2)*(x+3))

In [ ]:
# simplify uses trigonometric identities
simplify(sin(a)**2 + cos(a)**2)

In [ ]:
simplify(cos(x)/sin(x))

### apart and together

To manipulate symbolic expressions of fractions, we can use the `apart` and `together` functions:

In [ ]:
f1 = 1/((a+1)*(a+2))

In [ ]:
f1

In [ ]:
apart(f1)

In [ ]:
f2 = 1/(a+2) + 1/(a+3)

In [ ]:
f2

In [ ]:
together(f2)

Simplify usually combines fractions but does not factor: 

In [ ]:
simplify(f2)

## Calculus

In addition to algebraic manipulations, the other main use of CAS is to do calculus, like derivatives and integrals of algebraic expressions.

### Differentiation

Differentiation is usually simple. Use the `diff` function. The first argument is the expression to take the derivative of, and the second argument is the symbol by which to take the derivative:

In [ ]:
y

In [ ]:
diff(y**2, x)

For higher order derivatives we can do:

In [ ]:
diff(y**2, x, x)

In [ ]:
diff(y**2, x, 2) # same as above

To calculate the derivative of a multivariate expression, we can do:

In [ ]:
x, y, z = symbols("x,y,z")

In [ ]:
f = sin(x*y) + cos(y*z)

$\frac{d^3f}{dxdy^2}$

In [ ]:
diff(f, x, 1, y, 2)

## Integration

Integration is done in a similar fashion:

In [ ]:
f

In [ ]:
integrate(f, x)

By providing limits for the integration variable we can evaluate definite integrals:

In [ ]:
integrate(f, (x, -1, 1))

and also improper integrals

In [ ]:
integrate(exp(-x**2), (x, -oo, oo))

Remember, `oo` is the SymPy notation for inifinity.

### Sums and products

We can evaluate sums and products using the functions: 'Sum'

In [ ]:
n = Symbol("n")

In [ ]:
Sum(1/n**2, (n, 1, 10))

In [ ]:
Sum(1/n**2, (n,1, 10)).evalf()

In [ ]:
Sum(1/n**2, (n, 1, oo)).evalf()

Products work much the same way:

In [ ]:
Product(n, (n, 1, 10)) # 10!

## Limits

Limits can be evaluated using the `limit` function. For example, 

In [ ]:
limit(sin(x)/x, x, 0)

We can use 'limit' to check the result of derivation using the `diff` function:

In [ ]:
f

In [ ]:
diff(f, x)

$\displaystyle \frac{\mathrm{d}f(x,y)}{\mathrm{d}x} = \frac{f(x+h,y)-f(x,y)}{h}$

In [ ]:
h = Symbol("h")

In [ ]:
limit((f.subs(x, x+h) - f)/h, h, 0)

OK!

We can change the direction from which we approach the limiting point using the `dir` keywork argument:

In [ ]:
limit(1/x, x, 0, dir="+")

In [ ]:
limit(1/x, x, 0, dir="-")

## Series

Series expansion is also one of the most useful features of a CAS. In SymPy we can perform a series expansion of an expression using the `series` function:

In [ ]:
series(exp(x), x)

By default it expands the expression around $x=0$, but we can expand around any value of $x$ by explicitly include a value in the function call:

In [ ]:
series(exp(x), x, 1)

And we can explicitly define to which order the series expansion should be carried out:

In [ ]:
series(exp(x), x, 1, 10)

The series expansion includes the order of the approximation, which is very useful for keeping track of the order of validity when we do calculations with series expansions of different order:

In [ ]:
s1 = cos(x).series(x, 0, 5)
s1

In [ ]:
s2 = sin(x).series(x, 0, 2)
s2

In [ ]:
expand(s1 * s2)

If we want to get rid of the order information we can use the `removeO` method:

In [ ]:
expand(s1.removeO() * s2.removeO())

But note that this is not the correct expansion of $\cos(x)\sin(x)$ to $5$th order:

In [ ]:
(cos(x)*sin(x)).series(x, 0, 6)

## Linear algebra

### Matrices

Matrices are defined using the `Matrix` class:

In [ ]:
m11, m12, m21, m22 = symbols("m11, m12, m21, m22")
b1, b2 = symbols("b1, b2")

In [ ]:
A = Matrix([[m11, m12],[m21, m22]])
A

In [ ]:
b = Matrix([[b1], [b2]])
b

With `Matrix` class instances we can do the usual matrix algebra operations:

In [ ]:
A**2

In [ ]:
A * b

And calculate determinants and inverses, and the like:

In [ ]:
A.det()

In [ ]:
A.inv()

## Solving equations

For solving equations and systems of equations we can use the `solve` function:

In [ ]:
solve(x**2 - 1, x)

In [ ]:
solve(x**4 - x**2 - 1, x)

System of equations:

In [ ]:
solve([x + y - 1, x - y - 1], [x,y])

In terms of other symbolic expressions:

In [ ]:
solve([x + y - a, x - y - c], [x,y])

## Quantum mechanics: noncommuting variables

How about non-commuting symbols? In quantum mechanics we need to work with noncommuting operators, and SymPy has a nice support for noncommuting symbols and even a subpackage for quantum mechanics related calculations!

In [ ]:
from sympy.physics.quantum import *

## States

We can define symbol states, kets and bras:

In [ ]:
Ket('psi')

In [ ]:
Bra('psi')

In [ ]:
u = Ket('0')
d = Ket('1')

a, b = symbols('alpha beta', complex=True)

In [ ]:
phi = a * u + sqrt(1-abs(a)**2) * d; phi

In [ ]:
Dagger(phi)

In [ ]:
Dagger(phi) * d

Use `qapply` to distribute a mutiplication:

In [ ]:
qapply(Dagger(phi) * d)

In [ ]:
qapply(Dagger(phi) * u)

### Operators

In [ ]:
A = Operator('A')
B = Operator('B')

Check if they are commuting!

In [ ]:
A * B == B * A

In [ ]:
expand((A+B)**3)

In [ ]:
c = Commutator(A,B)
c

We can use the `doit` method to evaluate the commutator:

In [ ]:
c.doit()

We can mix quantum operators with C-numbers:

In [ ]:
c = Commutator(a * A, b * B)
c

To expand the commutator, use the `expand` method with the `commutator=True` keyword argument:

In [ ]:
c = Commutator(A+B, A*B)
c.expand(commutator=True)

In [ ]:
Dagger(Commutator(A, B))

In [ ]:
ac = AntiCommutator(A,B)

In [ ]:
ac.doit()

#### Example: Quadrature commutator

Let's look at the commutator of the electromagnetic field quadatures $x$ and $p$. We can write the quadrature operators in terms of the creation and annihilation operators as:

$\displaystyle x = (a + a^\dagger)/\sqrt{2}$

$\displaystyle p = -i(a - a^\dagger)/\sqrt{2}$


In [ ]:
X = (A + Dagger(A))/sqrt(2)
X

In [ ]:
P = -I * (A - Dagger(A))/sqrt(2)
P

Let's expand the commutator $[x,p]$

In [ ]:
Commutator(X, P).expand(commutator=True).expand(commutator=True)

Here we see directly that the well known commutation relation for the quadratures

$[x,p]=i$

is a directly related to

$[A, A^\dagger]=1$ 

(which SymPy does not know about, and does not simplify).

For more details on the quantum module in SymPy, see:

* http://docs.sympy.org/0.7.2/modules/physics/quantum/index.html
* http://nbviewer.ipython.org/urls/raw.github.com/ipython/ipython/master/docs/examples/notebooks/sympy_quantum_computing.ipynb

## Further reading

* http://sympy.org/en/index.html - The SymPy projects web page.
* https://github.com/sympy/sympy - The source code of SymPy.
* http://live.sympy.org - Online version of SymPy for testing and demonstrations.

## Versions

In [ ]:
%reload_ext version_information

%version_information numpy, sympy